### I want to make a map with only the regions I might have time to visit, and I want to start some of those regions from Kwik Trip where I can stay with family instead of my local Dunkin.

In [1]:
from config import maps_key, home_addr, alt_start_addr
from shared import *
import numpy as np
import pandas as pd
import requests
from sklearn import cluster

In [2]:
shops = pd.read_csv("input_data/geocoded_shops.csv")
shops

,region,shop name,address,lat,lng
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834


In [3]:
home_lat, home_lng = get_lat_lng(home_addr, maps_key)

alt_start_lat, alt_start_lng = get_lat_lng(alt_start_addr, maps_key)

### First, recluster data for only the regions I want to complete

In [4]:
shops = shops[shops["region"].isin(["metro", "east central", "northeast", "southeast", "west central"])]
shops

,region,shop name,address,lat,lng
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834


In [5]:
coords = ";".join(shops[["lng", "lat"]].apply(lambda row: ','.join(row.values.astype(str)), axis=1))
coords

'-93.0078652,45.0855867;-93.02603640000001,45.24449480000001;-93.38845620000001,45.1981781;-93.204791,44.7942112;-93.1250426,44.7373301;-93.1942715,45.0362424;-93.5209848,46.11646210000001;-94.3001749,45.313569799999996;-94.2068809,45.55924659999999;-93.5638344,45.3263943;-92.8846361,46.124488299999996;-93.553919,45.211043;-90.4445296,47.724412799999996;-92.0323471,46.827637200000005;-92.867063,46.8785185;-91.2991293,47.2589152;-92.45949770000001,46.746570399999996;-95.3780526,45.88617429999999;-94.5283382,45.1269287;-95.538025,44.8114027;-94.9442539,45.3011018;-94.36927790000001,44.896660600000004;-94.7148875,45.3821829;-91.6832632,44.055571;-93.2481113,43.651288799999996;-93.7253723,43.74428169999999;-92.4839857,44.004667100000006;-92.266553,44.449163399999996;-92.0354511,44.3843299'

In [6]:
def get_names(shops, region=None):
    names = []
    
    if region:
        s = shops[shops["region"] == region]
    else:
        s = shops

    for i, r in s.iterrows():
        names.append(r['shop name'])
    
    return names


In [7]:
place_names = get_names(shops)
place_names

['Affinity for Quilts',
 'Country Loft Quilt & Design',
 "Millie P's Quilt Shop",
 'Quilt Cove',
 "Quilter's Haven",
 'Twin Cities Quilting',
 'Country Caboose Quilts',
 'Gone to Pieces Quilt Shop',
 "Gruber's Quilt Shop",
 'The Noble Quilter',
 'Quarry Quilts & Yarns',
 'Quilted Treasures',
 '47 Degrees North Quilt Shop',
 'Hannah Johnson Fabrics',
 'Hingeley Road Quilt Shop',
 'Quilt Corner',
 'Quilted Dog Quilt Shop',
 "Dawn's Quilt Shop",
 "DeAnn's Country Village Shoppe",
 "Heather's Book Nook & Sew Much More",
 'Paisley Threads',
 'Quilt Haven on Main  ',
 'Sweetwater Cotton shoppe',
 'Bluffview Quilt Shop',
 'Calico Hutch',
 'The Humble Heart',
 'Pine Needles Quilt & Sew',
 'Rather Bee Quilting',
 "What's Brewing Quilt Shop"]

### Use the osrm api to get a distance matrix

In [8]:
matrix = requests.get(f"http://router.project-osrm.org/table/v1/driving/{coords}")

In [9]:
matrix

<Response [200]>

In [10]:
res = matrix.json()
res

{'code': 'Ok',
 'durations': [[0,
   1188.9,
   1928.2,
   2224.2,
   2556.6,
   1130.4,
   7313.1,
   6170.1,
   5428.5,
   2949.2,
   5054.1,
   2638.8,
   16866.3,
   9720.1,
   10716.3,
   13287.8,
   8395.5,
   9540.4,
   7278.7,
   11314.2,
   8315.8,
   6407.7,
   7308.4,
   9261.5,
   7698.5,
   8979.7,
   6501.2,
   5880.1,
   7148],
  [1243.9,
   0,
   1989,
   2913.3,
   3245.7,
   1473.7,
   6769.5,
   6235.7,
   5494.1,
   3010,
   4188.3,
   2704.4,
   16000.5,
   8854.3,
   9850.5,
   12422,
   7529.7,
   9606,
   7706.4,
   11853.4,
   8381.4,
   6752.9,
   7374,
   9961.4,
   8387.6,
   9668.8,
   7190.3,
   6580,
   7847.9],
  [1997.9,
   2056.9,
   0,
   3052.1,
   3528.8,
   1362.5,
   5614.3,
   4924.4,
   4157.7,
   1250.4,
   5922.1,
   1121.1,
   17734.3,
   10588.1,
   11584.3,
   14155.8,
   9263.5,
   8294.7,
   6450.9,
   10700.3,
   7070.1,
   5623,
   6062.7,
   10244.5,
   8276.1,
   9557.3,
   7473.4,
   6863.1,
   8131],
  [2282,
   2950.1,
   3078.1,
 

In [11]:
df = pd.DataFrame(res["durations"], columns=place_names, index=place_names)
df

,Affinity for Quilts,Country Loft Quilt & Design,Millie P's Quilt Shop,Quilt Cove,Quilter's Haven,Twin Cities Quilting,Country Caboose Quilts,Gone to Pieces Quilt Shop,Gruber's Quilt Shop,The Noble Quilter,...,Heather's Book Nook & Sew Much More,Paisley Threads,Quilt Haven on Main,Sweetwater Cotton shoppe,Bluffview Quilt Shop,Calico Hutch,The Humble Heart,Pine Needles Quilt & Sew,Rather Bee Quilting,What's Brewing Quilt Shop
Affinity for Quilts,0.0,1188.9,1928.2,2224.2,2556.6,1130.4,7313.1,6170.1,5428.5,2949.2,...,11314.2,8315.8,6407.7,7308.4,9261.5,7698.5,8979.7,6501.2,5880.1,7148.0
Country Loft Quilt & Design,1243.9,0.0,1989.0,2913.3,3245.7,1473.7,6769.5,6235.7,5494.1,3010.0,...,11853.4,8381.4,6752.9,7374.0,9961.4,8387.6,9668.8,7190.3,6580.0,7847.9
Millie P's Quilt Shop,1997.9,2056.9,0.0,3052.1,3528.8,1362.5,5614.3,4924.4,4157.7,1250.4,...,10700.3,7070.1,5623.0,6062.7,10244.5,8276.1,9557.3,7473.4,6863.1,8131.0
Quilt Cove,2282.0,2950.1,3078.1,0.0,898.0,1902.8,8261.9,6698.1,6088.1,3898.0,...,9953.5,8975.4,5311.4,7968.0,8733.0,5756.7,7037.9,5638.8,5351.6,6619.5
Quilter's Haven,2657.0,3325.1,3509.0,892.1,0.0,2313.0,8893.9,7353.9,6743.9,4530.0,...,10609.3,9631.2,5967.2,8623.8,7930.7,6116.2,7397.4,4836.5,4549.3,5817.2
Twin Cities Quilting,1150.7,1477.6,1363.4,1816.4,2313.7,0.0,6748.3,5591.5,4849.9,2384.4,...,10616.7,7737.2,5485.9,6729.8,9216.7,7290.7,8571.9,6445.6,5835.3,7103.2
Country Caboose Quilts,7386.8,7199.2,5588.8,8327.1,8917.7,6751.4,0.0,6503.2,5148.8,4664.0,...,12851.6,8614.2,8879.2,7606.8,15633.4,13403.7,14684.9,12862.3,12252.0,13519.9
Gone to Pieces Quilt Shop,6225.6,6320.3,4893.4,6755.3,7408.0,5548.5,6535.2,0.0,1468.1,4329.4,...,7512.4,3581.6,2376.0,2529.1,14415.3,11831.9,10821.5,11644.2,11033.9,12301.8
Gruber's Quilt Shop,5525.1,5619.8,4134.7,6190.0,6842.7,4848.0,5226.3,1494.9,0.0,3431.2,...,7793.6,3556.2,3870.9,2548.8,13714.8,11266.6,12316.4,10943.7,10333.4,11601.3
The Noble Quilter,3008.6,3067.6,1210.6,3948.9,4539.5,2373.2,4662.2,4277.8,3389.9,0.0,...,10660.9,6423.5,6102.8,5416.1,11255.2,9025.5,10306.7,8484.1,7873.8,9141.7


In [12]:
df.to_csv("output_data/5_regions_distance_matrix.csv")

### Affinity Propogation

In [13]:
X = df.to_numpy()
X

array([[    0. ,  1188.9,  1928.2,  2224.2,  2556.6,  1130.4,  7313.1,
         6170.1,  5428.5,  2949.2,  5054.1,  2638.8, 16866.3,  9720.1,
        10716.3, 13287.8,  8395.5,  9540.4,  7278.7, 11314.2,  8315.8,
         6407.7,  7308.4,  9261.5,  7698.5,  8979.7,  6501.2,  5880.1,
         7148. ],
       [ 1243.9,     0. ,  1989. ,  2913.3,  3245.7,  1473.7,  6769.5,
         6235.7,  5494.1,  3010. ,  4188.3,  2704.4, 16000.5,  8854.3,
         9850.5, 12422. ,  7529.7,  9606. ,  7706.4, 11853.4,  8381.4,
         6752.9,  7374. ,  9961.4,  8387.6,  9668.8,  7190.3,  6580. ,
         7847.9],
       [ 1997.9,  2056.9,     0. ,  3052.1,  3528.8,  1362.5,  5614.3,
         4924.4,  4157.7,  1250.4,  5922.1,  1121.1, 17734.3, 10588.1,
        11584.3, 14155.8,  9263.5,  8294.7,  6450.9, 10700.3,  7070.1,
         5623. ,  6062.7, 10244.5,  8276.1,  9557.3,  7473.4,  6863.1,
         8131. ],
       [ 2282. ,  2950.1,  3078.1,     0. ,   898. ,  1902.8,  8261.9,
         6698.1,  6088.

In [14]:
clustering = cluster.AffinityPropagation(random_state=7).fit(X)
clustering

AffinityPropagation(random_state=7)

In [15]:
clustering.labels_

array([0, 0, 0, 0, 0, 0, 2, 3, 3, 0, 2, 0, 1, 2, 2, 1, 2, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4])

In [16]:
new_groups_df = shops.copy()

In [17]:
new_groups_df["affinity_cluster"] = clustering.labels_
new_groups_df

,region,shop name,address,lat,lng,affinity_cluster
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865,0
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036,0
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456,0
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791,0
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043,0
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271,0
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985,2
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175,3
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881,3
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834,0


In [18]:
new_groups_df.to_csv("output_data/5_regions_affinity_clusters.csv", index=False)

### Mapping new clusters

In [19]:
shops = pd.read_csv("output_data/5_regions_affinity_clusters.csv")
shops

,region,shop name,address,lat,lng,affinity_cluster
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865,0
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036,0
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456,0
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791,0
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043,0
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271,0
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985,2
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175,3
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881,3
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834,0


In [20]:
m, distances, durations = map_all_regions(shops, home_lat, home_lng, region_col="affinity_cluster")

In [21]:
m

In [22]:
distances

{0: 154.70633924465932,
 2: 382.7071967390359,
 3: 415.15503249779414,
 1: 516.8616327190028,
 4: 382.3925335851964}

In [23]:
durations

{0: 3.5764444444444448,
 2: 7.96611111111111,
 3: 9.153527777777777,
 1: 9.825027777777779,
 4: 8.164194444444444}

In [24]:
rm = add_all_region_markers(shops, existing_map=m)

In [25]:
rm

### Now, I want to start 4 of the trips from home, and one trip from a different location.

In [26]:
home_shops = shops[shops["affinity_cluster"].isin([0,1,2,3])]
home_shops

,region,shop name,address,lat,lng,affinity_cluster
0,metro,Affinity for Quilts,"2199 4th St, White Bear Lake, MN 55110",45.085587,-93.007865,0
1,metro,Country Loft Quilt & Design,"15161 Feller St NE, Forest Lake, MN 55025",45.244495,-93.026036,0
2,metro,Millie P's Quilt Shop,"219 E Main St, Anoka, MN 55303",45.198178,-93.388456,0
3,metro,Quilt Cove,"1960 Cliff Lake Rd, Suite 134, Eagan, MN 55122",44.794211,-93.204791,0
4,metro,Quilter's Haven,"2930 146th St, Suite 108, Rosemount, MN 55068",44.737330,-93.125043,0
5,metro,Twin Cities Quilting,"550 County Rd D, New Brighton, MN 55112",45.036242,-93.194271,0
6,east central,Country Caboose Quilts,"108 South Main St, Wahkon, MN 56386",46.116462,-93.520985,2
7,east central,Gone to Pieces Quilt Shop,"70 S Main, Kimball, MN 55353",45.313570,-94.300175,3
8,east central,Gruber's Quilt Shop,"310 4th Ave NE, Waite Park, MN 56387",45.559247,-94.206881,3
9,east central,The Noble Quilter,"19570 Holt St NW, Elk River, MN 55330",45.326394,-93.563834,0


In [35]:
m, distances, durations = map_all_regions(home_shops, home_lat, home_lng, region_col="affinity_cluster")

In [36]:
m

In [29]:
distances

{0: 154.70633924465932,
 2: 382.7071967390359,
 3: 415.15503249779414,
 1: 516.8616327190028}

In [30]:
durations

{0: 3.5764444444444448,
 2: 7.96611111111111,
 3: 9.153527777777777,
 1: 9.825027777777779}

In [37]:
away_shops = shops[shops["affinity_cluster"] == 4]
away_shops

,region,shop name,address,lat,lng,affinity_cluster
23,southeast,Bluffview Quilt Shop,"1671 W 5th St, Winona, MN 55987",44.055571,-91.683263,4
24,southeast,Calico Hutch,"20520 810th Ave, Hayward, MN 56043",43.651289,-93.248111,4
25,southeast,The Humble Heart,"49 S Broadway, Wells, MN 56097",43.744282,-93.725372,4
26,southeast,Pine Needles Quilt & Sew,"1300 Salem Rd SW, Rochester, MN 55902",44.004667,-92.483986,4
27,southeast,Rather Bee Quilting,"106 S Lakeshore Dr, Lake City, MN 55041",44.449163,-92.266553,4
28,southeast,What's Brewing Quilt Shop,"330 2nd St W, Wabasha 55981",44.384330,-92.035451,4


In [38]:
new_m, distances, durations = map_all_regions(away_shops, alt_start_lat, alt_start_lng, region_col="affinity_cluster", existing_map=m)

In [39]:
new_m

In [40]:
distances

{4: 285.7529794822723}

In [41]:
durations

{4: 5.988138888888888}

In [42]:
rm = add_all_region_markers(shops, existing_map=new_m)

In [43]:
rm

In [45]:
rm.save("output_data/5_trips_index.html")